In [1]:
%cd ..

/Users/mateoibarguen/Desktop/CSC 482/complementary_fact_finder


In [4]:
from gensim.corpora import WikiCorpus, MmCorpus, Dictionary
from nltk.tokenize import sent_tokenize, word_tokenize
import os
import random
import codecs
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel, TfidfModel, LsiModel
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import pickle
import json
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.similarities import MatrixSimilarity
from gensim.models.coherencemodel import CoherenceModel
from src.topic_models import *

In [48]:
# !pip install pyLDAvis

In [39]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [272]:
with open ('data/token_sents.pkl', 'rb') as fp:
    wiki_tokens = pickle.load(fp)

In [273]:
with open ('data/sents.pkl', 'rb') as fp:
    wiki_sents = pickle.load(fp)

In [246]:
from gensim.models import wrappers

In [ ]:
doc = student_tokens[2]
vec_bow = wiki_tfidf[wiki_dict.doc2bow(doc)]
vec_lsi = lsi[vec_bow]
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [310]:
class WikiModel:
    def __init__(self, wiki_tokens_path='data/token_sents.pkl', wiki_sents_path='data/sents.pkl', student_tokens_path='data/data.json'):
        self.wiki_tokens = self.load_wiki(wiki_tokens_path)
        self.wiki_sents = self.load_wiki(wiki_sents_path)
        self.wiki_dict = Dictionary(wiki_tokens)
        self.wiki_corpus = [wiki_dict.doc2bow(t) for t in wiki_tokens]
        self.student_tokens = self.load_student(student_tokens_path)
        
    def load_wiki(self, filepath):
        with open(filepath, 'rb') as f:
            return pickle.load(f)
        
    def load_student(self, filepath):
        with open(filepath, 'rb') as f:
            return json.load(f)

In [355]:
class LsiWikiModel(WikiModel):
    def __init__(self, num_topics=100):
        super().__init__()
        self.wiki_tfidf = TfidfModel(self.wiki_corpus, id2word=self.wiki_dict)
        self.wiki_tfidf_corpus = self.wiki_tfidf[self.wiki_corpus]
        self.lsi = self.compute_lsi(num_topics)
        self.lsi_index = MatrixSimilarity(self.lsi[self.wiki_tfidf_corpus])
        
    def compute_lsi(self, num_topics=None):
        lsi = LsiModel(self.wiki_tfidf_corpus, num_topics=num_topics, id2word=self.wiki_dict)
        return lsi
    
    def predict_lsi(self, text, num_results=10, print_results=True):
        vec_bow = self.wiki_tfidf[self.wiki_dict.doc2bow(text)]
        vec_lsi = self.lsi[vec_bow]
        sims = self.lsi_index[vec_lsi]
        sorted_sims = sorted(enumerate(sims), key=lambda item: -item[1])
        preds = [(index, score) for index, score in sorted_sims[:num_results]]
        if print_results:
            print(' '.join(text))
            print('\n\n')
            print('Results: \n')
            for i in self.string_preds(preds):
                print(f'Score: {round(i[1], 3)}')
                print(i[0])
        return preds
    
    def string_preds(self, preds):
        string_results = [(self.wiki_sents[pred[0]], pred[1]) for pred in preds]
        return string_results
    
    def predict_lsi_index(self, student_index, num_results=10, print_results=True):
        text = self.student_tokens[student_index]
        return self.predict_lsi(text, num_results, print_results)


In [356]:
class LdaWikiModel(WikiModel):
    def __init__(self, num_topics=100):
        super().__init__()
        self.lda = self.compute_lda(num_topics)
        self.lda_index = MatrixSimilarity(self.lda[self.wiki_corpus])
        
    def compute_lda(self, num_topics=None):
        lda = LdaModel(self.wiki_corpus, num_topics=num_topics, id2word=self.wiki_dict)
        return lda
    
    def predict_lda(self, text, num_results=10, print_results=True):
        vec_bow = self.wiki_dict.doc2bow(text)
        vec_lda = self.lda[vec_bow]
        sims = self.lda_index[vec_lda]
        sorted_sims = sorted(enumerate(sims), key=lambda item: -item[1])
        preds = [(index, score) for index, score in sorted_sims[:num_results]]
        if print_results:
            print(' '.join(text))
            print('\n\n')
            print('Results: \n')
            for i in self.string_preds(preds):
                print(f'Score: {round(i[1], 3)}')
                print(i[0])
        return preds
    
    def string_preds(self, preds):
        string_results = [(self.wiki_sents[pred[0]], pred[1]) for pred in preds]
        return string_results
    
    def predict_lda_index(self, student_index, num_results=10, print_results=True):
        text = self.student_tokens[student_index]
        return self.predict_lda(text, num_results, print_results)


In [7]:
from src.topic_models import *

In [8]:
lda_wiki = LdaWikiModel()

In [21]:
len(lda_wiki.wiki_sents)

14391

In [25]:
lda_wiki.predict_lda_index(student_index=10, num_results=20)

dog know baby dog call pup puppy dog call puppy one year old dog sometimes referred man best friend kept domestic pet usually loyal like around human dog breed earliest dog breeder used wolf create domestic dog beginning human purposefully bred dog perform various task sleek greyhound type bred chase fleet football prey huge mastiff example dog bred specific job dog save day dog save day show dog police dog one water get dog save helper see even help open door help walk home home see dog lot like friend sometimes police dog wear gear dog help dog help help bye see get blind dog helper help lot help open door help keep away man take money well dog help come time smart like give money dog went get drink dog come back see dog drink want well teach dog dog become pet new evidene dog probably evolved wolf single location 20000 40000 year ago study suggests looking rate change dna oldest specimen scientist able place timing domestication dog 20000 40000 year ago train dog frist need give dog

[(5675, 0.60639644),
 (8129, 0.5836093),
 (5677, 0.55745506),
 (2841, 0.5446346),
 (3362, 0.5431265),
 (4130, 0.53738976),
 (3368, 0.53556734),
 (1074, 0.51459587),
 (1386, 0.5126204),
 (2423, 0.51121175),
 (13951, 0.5057534),
 (2148, 0.4985434),
 (1493, 0.4931624),
 (1490, 0.48855722),
 (14268, 0.4831299),
 (1679, 0.47999477),
 (9835, 0.47652665),
 (2140, 0.4760283),
 (2150, 0.4760283),
 (3112, 0.4760283)]

In [10]:
lsi_wiki = LsiWikiModel()

In [ ]:
"My dog is cool. "

In [15]:
lsi_wiki.predict_lsi(['dog', 'cat'])

dog cat



Results: 

Score: 0.8650000095367432
('Cat', 'Cats which are not specific breeds can be referred to as domestic shorthair DSH or domestic longhair DLH.')
Score: 0.8090000152587891
('Grey wolf', 'It is the ancestor of the domestic dog.')
Score: 0.8059999942779541
('Cat', 'The big cats and wild cats are not tame and can be very dangerous.')
Score: 0.7630000114440918
('Lion', 'Compared to other cats lions are social.')
Score: 0.6930000185966492
('Cat', 'Domestic cats are often called house cats when kept as indoor pets.')
Score: 0.6759999990463257
('Cat', 'Domestic cats are found in shorthair longhair and hairless breeds.')
Score: 0.6620000004768372
('Cat', 'Felines are usually called either big cats or small cats.')
Score: 0.6309999823570251
('Cat', '\n\nCats also called domestic cats Felis catus are small carnivorous meateating mammals of the family Felidae.')
Score: 0.578000009059906
('Dog', 'Dogs often have jobs including as police dogs army dogs assistance dogs fire dogs m

[(2041, 0.8649106),
 (13679, 0.8091616),
 (2046, 0.80628717),
 (4735, 0.7630837),
 (2030, 0.69328415),
 (2040, 0.6757196),
 (2043, 0.66165376),
 (2029, 0.63088405),
 (2845, 0.5783287),
 (4714, 0.5479054)]

In [20]:
lsi_wiki.predict_lsi_index(student_index=10, num_results=20)

dog know baby dog call pup puppy dog call puppy one year old dog sometimes referred man best friend kept domestic pet usually loyal like around human dog breed earliest dog breeder used wolf create domestic dog beginning human purposefully bred dog perform various task sleek greyhound type bred chase fleet football prey huge mastiff example dog bred specific job dog save day dog save day show dog police dog one water get dog save helper see even help open door help walk home home see dog lot like friend sometimes police dog wear gear dog help dog help help bye see get blind dog helper help lot help open door help keep away man take money well dog help come time smart like give money dog went get drink dog come back see dog drink want well teach dog dog become pet new evidene dog probably evolved wolf single location 20000 40000 year ago study suggests looking rate change dna oldest specimen scientist able place timing domestication dog 20000 40000 year ago train dog frist need give dog

[(13679, 0.62648535),
 (2985, 0.59825754),
 (2850, 0.59361124),
 (1474, 0.5624349),
 (13485, 0.5611255),
 (2851, 0.5523321),
 (4976, 0.5513568),
 (13160, 0.543472),
 (4632, 0.5265628),
 (25, 0.52626187),
 (5675, 0.52595127),
 (391, 0.5259104),
 (13170, 0.5246036),
 (5818, 0.52324283),
 (5292, 0.522913),
 (211, 0.52177334),
 (2856, 0.5146035),
 (2849, 0.5137134),
 (13650, 0.5109728),
 (1476, 0.5089854)]

In [300]:
class WikiModel:
    def __init__(self, wiki_tokens=wiki_tokens, wiki_sents=wiki_sents):
        self.wiki_tokens = wiki_tokens
        self.wiki_sents = wiki_sents
        self.wiki_dict = Dictionary(wiki_tokens)
        self.wiki_corpus = [wiki_dict.doc2bow(t) for t in wiki_tokens]
        self.wiki_tfidf = TfidfModel(self.wiki_corpus, id2word=self.wiki_dict)
        self.wiki_tfidf_corpus = self.wiki_tfidf[self.wiki_corpus]
        self.lsi = self.compute_lsi(100)
        self.lsi_index = MatrixSimilarity(self.lsi[self.wiki_tfidf_corpus])
        self.lda = self.compute_lda(100)
        self.lda_index = MatrixSimilarity(self.lda[self.wiki_corpus])
    
    def compute_lsi(self, num_topics=None):
        lsi = LsiModel(self.wiki_tfidf_corpus, num_topics=num_topics, id2word=self.wiki_dict)
        return lsi

    def compute_lda(self, num_topics=None):
        lda = LdaModel(self.wiki_corpus, num_topics=num_topics, id2word=self.wiki_dict)
        return lda
    
    def predict_lsi(self, text, num_results):
        print(text)
        print('\n')
        vec_bow = self.wiki_tfidf[self.wiki_dict.doc2bow(text)]
        vec_lsi = self.lsi[vec_bow]
        sims = self.lsi_index[vec_lsi]
        sorted_sims = sorted(enumerate(sims), key=lambda item: -item[1])
        for i in sorted_sims[:100]:
            print(self.wiki_sents[i[0]])
        

In [ ]:
class LsiWikiModel(WikiModel):
    def __init__(self)

In [301]:
topic_model = WikiModel()

In [302]:
topic_model.predict_lsi(student_tokens[0], 5)

['gold', 'rush', 'introduction', 'beginning', 'year', '1848', 'life', 'changing', 'gold', 'rush', 'begun', 'james', 'marshall', 'assigned', 'build', 'sawmill', 'john', 'sutters', 'workersin', 'process', 'found', 'three', 'gold', 'nugget', 'showed', 'gold', 'john', 'sutterjohn', 'sutter', 'wanted', 'keep', 'secret', 'sam', 'brannan', 'spread', 'word', 'immigrant', 'gold', 'said', 'rosie', 'stripped', 'wheat', 'yelped', 'oooohhhh', 'people', 'wanted', 'gold', 'rush', 'president', 'polk', 'said', 'gold', 'california', 'gold', 'rush', 'item', 'stare', 'ball', 'favorite', 'people', 'item', 'gold', 'rush', 'like', 'gamesclothes', 'toolswhich', 'let', 'know', 'time', 'period', 'conclusion']


('January 24', ' 1848  California gold rush James W Marshall finds gold at Sutters Mill near Sacramento.')
('United States dollar', 'Some are silver colored and some are gold colored.')
('December 5', '1848  California gold rush In a message before the US Congress US President James K Polk confirms that 

In [259]:
lsi = topic_model.compute_lsi(20)

In [262]:
# cm = CoherenceModel(lsi, corpus = topic_model.wiki_corpus, dictionary=topic_model.wiki_dict, text)

In [256]:
# model_list, coherence_values = topic_model.compute_coherence_values(dictionary=topic_model.wiki_dict, corpus=topic_model.wiki_corpus, texts=topic_model.wiki_tokens, start=2, limit=40, step=6)

In [289]:
wiki_dict = Dictionary(wiki_tokens)
wiki_corpus = [wiki_dict.doc2bow(t) for t in wiki_tokens]

In [290]:
wiki_tfidf = TfidfModel(wiki_corpus, id2word=wiki_dict)
wiki_tfidf_corpus = wiki_tfidf[wiki_corpus]

In [291]:
lsi = LdaModel(wiki_tfidf_corpus, num_topics=100, id2word=wiki_dict)

In [226]:
lsi.log_perplexity(wiki_tfidf_corpus)

-201.1761474609371

In [166]:
index = MatrixSimilarity(lsi[wiki_tfidf_corpus])

In [330]:
lsi = LdaModel(wiki_corpus, id2word=wiki_dict, num_topics=100)
index = MatrixSimilarity(lsi[wiki_corpus])

### Student tokens exploration

In [14]:
with open('data/data.json', 'r') as fb:
    student_tokens = json.load(fb)

In [68]:
student_dict = Dictionary(student_tokens)
student_corpus = [student_dict.doc2bow(s) for s in student_tokens]

In [64]:
for c in student_corpus:
    print(c)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 9), (14, 1), (15, 1), (16, 2), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 5), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1)]
[(9, 2), (19, 5), (20, 1), (46, 2), (54, 1), (55, 3), (56, 1), (57, 4), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 5), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 6), (72, 8), (73, 3), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 8), (80, 1), (81, 1), (82, 1), (83, 15), (84, 1), (85, 1), (86, 1), (87, 9), (88, 3), (89, 4), (90, 3), (91, 1), (92, 1), (93, 15), (94, 1), (95, 7), (96, 1), (97, 1), (98, 1), (99, 1), (100, 6), (101, 4), (102, 1), (103, 2), (104, 1), (105, 5), (106, 1), (

In [52]:
tfidf = TfidfModel(student_corpus, id2word=student_dict) 

In [55]:
student_corpus_tfidf = tfidf[student_corpus]

In [56]:
student_corpus_tfidf

In [100]:
student_lda = LdaModel(student_corpus, num_topics=10, id2word=student_dict)

In [101]:
student_lda.print_topics(num_words=10)

[(0,
  '0.012*"get" + 0.009*"game" + 0.009*"know" + 0.008*"like" + 0.008*"one" + 0.007*"also" + 0.006*"play" + 0.005*"shark" + 0.005*"dont" + 0.005*"dog"'),
 (1,
  '0.030*"wolf" + 0.010*"gray" + 0.009*"pack" + 0.008*"get" + 0.008*"like" + 0.008*"one" + 0.007*"also" + 0.007*"dont" + 0.007*"know" + 0.006*"animal"'),
 (2,
  '0.015*"dog" + 0.010*"play" + 0.010*"game" + 0.009*"know" + 0.009*"also" + 0.008*"need" + 0.007*"like" + 0.006*"people" + 0.006*"get" + 0.006*"different"'),
 (3,
  '0.011*"animal" + 0.010*"like" + 0.010*"one" + 0.010*"cat" + 0.009*"wolf" + 0.008*"also" + 0.008*"lot" + 0.008*"know" + 0.008*"dog" + 0.007*"eat"'),
 (4,
  '0.014*"like" + 0.010*"wolf" + 0.010*"one" + 0.009*"get" + 0.008*"know" + 0.008*"dog" + 0.007*"many" + 0.007*"also" + 0.007*"eat" + 0.006*"thing"'),
 (5,
  '0.021*"wolf" + 0.010*"like" + 0.010*"gray" + 0.009*"also" + 0.009*"get" + 0.008*"dog" + 0.008*"one" + 0.007*"play" + 0.007*"know" + 0.007*"need"'),
 (6,
  '0.018*"dog" + 0.008*"get" + 0.007*"one" + 0.

In [74]:
student_lda[student_corpus[0]]

[(0, 0.9874949)]

In [35]:
# student_tokens[34]

In [44]:
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(student_lda, student_corpus, student_dict)
# vis

In [45]:
len(student_tokens)

210

In [110]:
doc = ['wolf', 'cool']
test_term = student_lda.id2word.doc2bow(doc)
doc_topics = student_lda.get_document_topics(test_term, minimum_probability=0.0002)
#term_topics = student_lda.get_term_topics(ter)

In [111]:
doc_topic

[(5, 0.6998773)]

In [97]:
test_term

[(130, 1), (306, 1)]

In [113]:
doc = student_tokens[0]

In [117]:
test_term = student_lda.id2word.doc2bow(doc)
doc_topics =student_lda.get_document_topics(test_term, minimum_probability=0.20)

In [120]:
doc_topics

[(3, 0.987487)]

In [149]:
# student_tokens[0]

In [188]:
student_tokens[2]

['guide',
 'rock',
 'mineral',
 '4000',
 'different',
 'type',
 'mineral',
 'numerous',
 'rare',
 'rock',
 'mineral',
 'also',
 'ton',
 'common',
 'rock',
 'mineral',
 'every',
 'month',
 'one',
 'sometimes',
 'two',
 'birthstones',
 'silver',
 'gold',
 'hope',
 'enjoy',
 'guide',
 'rock',
 'mineral',
 'first',
 'four',
 'rare',
 'rock',
 'diamond',
 'ruby',
 'sapphire',
 'emerald',
 'antimatter',
 'cost',
 '625',
 'trillion',
 'per',
 'gram',
 'next',
 'rock',
 'mineral',
 'common',
 'rock',
 'mineral',
 'common',
 'like',
 'serpentine',
 'serpentine',
 'california',
 'state',
 'rock',
 'common',
 'california',
 'also',
 'birthstones',
 'know',
 'month',
 'one',
 'birthstone',
 'like',
 'october',
 'opal',
 'tourmaline',
 'emerald',
 'may',
 'popular',
 'one',
 'reason',
 'popular',
 'pretty',
 'finally',
 'silver',
 'gold',
 'silver',
 'cost',
 '1550',
 'per',
 'ounce',
 'gold',
 'cost',
 '128870',
 'per',
 'ounce',
 'gold',
 'worth',
 'per',
 'ounce',
 'silver',
 'also',
 'denser',


In [331]:
student_doc = student_tokens[0]
wiki_corpus[wiki_dict.doc2bow(student_doc)]

TypeError: list indices must be integers or slices, not list

In [294]:
def predict_student_doc(student_index):
    student_doc = student_tokens[student_index]
    print(' '.join(student_doc))
    print('\n\n')
    vec_bow = wiki_tfidf[wiki_dict.doc2bow(student_doc)]
    vec_lsi = lsi[vec_bow]
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    for i in sims[:10]:
        print('Score: ', i[1])
        print(wiki_sents[i[0]])
        print('\n')
    return 

In [295]:
predict_student_doc(60)

informational ondemand like stunt yes book snowboarding fun hit jump learn new thing steep slope hit jump snowboarding fun hit jump example mammoth ski resort tried heel grab face planted powder snow another example backflip accident powder jump one example 180 landed learn new thing snowboarding fun learn new thing example learned stop another example learned carve snow one example learned jump board conclusion snowboarding fun hit jump learn new thing steep slope think snowboarding fun fun think snowboarding fun



Score:  0.6948601
('Special relativity', 'The result was special relativity theory which brought together a new principle the constancy of the speed of light  and the previously established principle of relativity.')


Score:  0.69151306
('August 15', ' 1869  The Meiji Government in Japan establishes six new ministries including one for Shinto.')


Score:  0.6830139
('July 18', ' 1898  Marie and Pierre Curie announce the discovery of a new element and proposed to call it p

In [189]:
doc = student_tokens[2]
vec_bow = wiki_tfidf[wiki_dict.doc2bow(doc)]
vec_lsi = lsi[vec_bow]
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [190]:
sims

[(2906, 0.64762014),
 (13830, 0.6467161),
 (5716, 0.62135035),
 (3268, 0.61610115),
 (4017, 0.60542756),
 (2769, 0.58359087),
 (14009, 0.582927),
 (3658, 0.5827099),
 (590, 0.5715759),
 (14390, 0.56706244),
 (5642, 0.55757827),
 (8412, 0.5410841),
 (13725, 0.5406643),
 (616, 0.5378932),
 (13999, 0.52986586),
 (966, 0.5278363),
 (5938, 0.52774084),
 (14093, 0.5201176),
 (13647, 0.5199637),
 (1776, 0.50328875),
 (3628, 0.5031408),
 (3816, 0.5028924),
 (13926, 0.5013269),
 (4006, 0.5005419),
 (60, 0.49680296),
 (13924, 0.492888),
 (531, 0.49031886),
 (5625, 0.48906487),
 (5412, 0.48881304),
 (323, 0.4885143),
 (3871, 0.48699108),
 (3255, 0.485152),
 (3784, 0.4839411),
 (2905, 0.48117003),
 (5294, 0.4806291),
 (13377, 0.47932982),
 (5409, 0.47821534),
 (13990, 0.47720855),
 (4632, 0.47518128),
 (3007, 0.47397462),
 (2430, 0.47365868),
 (5744, 0.47243926),
 (7952, 0.47081646),
 (5631, 0.46991056),
 (2400, 0.46840382),
 (1563, 0.46755657),
 (2822, 0.46680132),
 (13820, 0.46575016),
 (13872, 

In [194]:
wiki_sents[3268]

('Video game',
 'Also in 2004 Nintendo released the Dual Screen DS which has two separate screens one of which is a touchscreen.')